In [2]:
import pandas as pd
import csv


# Data Preprocessing Steps:
# 1. Extracting unique values from rows with multiple values.
# 2. Creating tables containing extracted unique values as key-value pairs.
# 3. Expanding rows with multiple values into separate rows.
# 4 Map values


def create_dataframe_from_unique_values(unique_values):
    """
    Creates a DataFrame from a list of unique values with two columns: keys and values.

    Args:
    unique_values (list): A list of unique values.

    Returns:
    pd.DataFrame: A DataFrame with two columns - 'keys' and 'values'.
    """
    # Generate keys as a range of integers from 0 to the length of unique_values
    keys = range(len(list(unique_values)))

    # Create the DataFrame
    df = pd.DataFrame({
        'keys': keys,
        'values': unique_values
    })

    return df


def extract_unique_values(df, column):
    """
    Extracts and returns unique values from a specified column in a DataFrame.
    
    Args:
    df (pd.DataFrame): The DataFrame to process.
    column (str): The name of the column from which to extract unique values.

    Returns:
    set: A set of unique values.
    """
    unique_vals = set()
    # Iterate over non-null values and update the set with unique values
    for val in df[column].dropna():
        unique_vals.update(val.split(','))
    return list(unique_vals)

def expand(df, column_name):
    """
    Expands a DataFrame by splitting a specified column's multiple values into separate rows.

    Args:
    df (pd.DataFrame): The DataFrame to expand.
    column_name (str): The name of the column to expand.

    Returns:
    pd.DataFrame: An expanded DataFrame with separate rows for each value.
    """
    expanded_rows = []
    for _, row in df.iterrows():
        vals = row[column_name]
        exp_list = vals.split(',') if isinstance(vals, str) else [None]

        for val in exp_list:
            new_row = row.to_dict()
            new_row[column_name] = val
            expanded_rows.append(new_row)

    expanded_df = pd.DataFrame(expanded_rows)
    expanded_df.drop_duplicates(inplace=True)
    expanded_df.reset_index(drop=True, inplace=True)
    return expanded_df

def map_fun (data,df,column):
    
    # mapping function 
    
    data[column] = data[column] .map({v:i for i,v in zip(df['keys'],df['values'])})
    return data


## Loading the table

In [3]:
name_basics = pd.read_csv(r'data.tsv', delimiter='\t', nrows=10)
title_akas = pd.read_csv(r'data.tsv', delimiter='\t',nrows = 1000 )
title_basics = pd.read_csv(r'data.tsv', delimiter='\t',nrows = 10)
title_crew = pd.read_csv(r'title.crew.tsv\data.tsv', delimiter='\t',nrows = 10)
title_ratings = pd.read_csv(r'title.ratings.tsv\data.tsv', delimiter='\t',nrows = 10)

In [4]:
name_basics.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0050419,tt0031983,tt0053137,tt0072308"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0038355,tt0117057,tt0037382,tt0075213"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0056404,tt0054452,tt0049189,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0080455,tt0072562,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0069467,tt0083922,tt0050986,tt0050976"


In [5]:
def combine(df, column):
    ext_dict = extract_unique_values(df,column)
    ext_dict
    table = create_dataframe_from_unique_values(ext_dict)
    return table

## Profession Table

In [6]:
combine(name_basics, 'primaryProfession')

,keys,values
0,0,music_department
1,1,director
2,2,soundtrack
3,3,actor
4,4,producer
5,5,writer
6,6,miscellaneous
7,7,actress


## Genre Table

In [7]:
genre = combine(title_basics, 'genres')

In [8]:
genre.head()

,keys,values
0,0,Sport
1,1,Romance
2,2,Documentary
3,3,Animation
4,4,Short


## Language Table

In [9]:
combine(title_akas, 'language')

,keys,values
0,0,ja
1,1,en
2,2,\N
3,3,tr
4,4,cs
5,5,sr
6,6,sv
7,7,es
8,8,ru


## Person table 

In [10]:
# .to_csv('Person.csv')
Person = name_basics[['nconst', 'primaryName', 'birthYear', 'deathYear']]
Person.head()

,nconst,primaryName,birthYear,deathYear
0,nm0000001,Fred Astaire,1899,1987
1,nm0000002,Lauren Bacall,1924,2014
2,nm0000003,Brigitte Bardot,1934,\N
3,nm0000004,John Belushi,1949,1982
4,nm0000005,Ingmar Bergman,1918,2007


## MoviePerson Table

In [11]:
name_basics_knownForTitle = expand(name_basics, 'knownForTitles')
name_basics_knownForTitle[['nconst', 'knownForTitles']].head()


,nconst,knownForTitles
0,nm0000001,tt0050419
1,nm0000001,tt0031983
2,nm0000001,tt0053137
3,nm0000001,tt0072308
4,nm0000002,tt0038355


## Type Table

In [12]:
title_basics_unique_type = extract_unique_values(title_basics, 'titleType')
type = create_dataframe_from_unique_values(title_basics_unique_type)
type


,keys,values
0,0,movie
1,1,short


# Region Table 

In [13]:

region_unique_type = extract_unique_values(title_akas, 'region')
region = create_dataframe_from_unique_values(region_unique_type)
region.head()

,keys,values
0,0,GR
1,1,DK
2,2,NZ
3,3,FR
4,4,UY


## Movie Table 

In [14]:
title_basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
7,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short"
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
9,tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short"


In [15]:
# Mapping using the Type dataframe

map_fun(title_basics,type, 'titleType')[['tconst','titleType','primaryTitle','originalTitle','isAdult','startYear','endYear']]



,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear
0,tt0000001,1,Carmencita,Carmencita,0,1894,\N
1,tt0000002,1,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N
2,tt0000003,1,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N
3,tt0000004,1,Un bon bock,Un bon bock,0,1892,\N
4,tt0000005,1,Blacksmith Scene,Blacksmith Scene,0,1893,\N
5,tt0000006,1,Chinese Opium Den,Chinese Opium Den,0,1894,\N
6,tt0000007,1,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N
7,tt0000008,1,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N
8,tt0000009,0,Miss Jerry,Miss Jerry,0,1894,\N
9,tt0000010,1,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N


##  MovieGenre Table


In [16]:
title_basics

MovieG= expand(title_basics, 'genres')
MovieG
# Expand 

map_fun(MovieG,genre, 'genres')[['tconst','genres']].head()


,tconst,genres
0,tt0000001,2
1,tt0000001,4
2,tt0000002,3
3,tt0000002,4
4,tt0000003,3
